In [7]:
import jax
import jax.numpy as jnp
import numpy as np
from jax import grad
from jax import jit
import cvxopt # TODO: speed up via convex approach
from scipy.optimize import minimize

In [9]:
def hittingCost(h, v_t):
    def func(y):
        return h(y-v_t)
    return func

In [107]:
''' Hitting cost test: '''
def h(x): #TODO: replace with control function
    return np.sum(x**2)
v_t = 2.3
func = hittingCost(h, v_t)

''' Try different commands: '''
print(func(2)) #should be 0.09
print(func(2.3)) # should be 0
print(hittingCost(h, v_t)(2.4)) # should be 0.01

''' Try vectors '''
v_t = np.array([1, 2, 3, 4])
print(hittingCost(h, v_t)(np.array([1, 2, 2, 2]))) # should print 5

print("works!")

0.0899999999999999
0.0
0.010000000000000018
5
works!


In [16]:
def dist(v_t):
    def func(y):
        norm = np.linalg.norm(y-v_t)
        return (norm**2)/2
    return func

In [108]:
'''Distance (L2 norm) test: '''
v_t = 2.5
print(dist(v_t)(2)) #should print .125 (works)

''' Vector test'''
v_t = np.array([1,2,3,4])
print(dist(v_t)(np.array([1,2,2,2]))) #should print 2.5 (works)
print("works!")

0.125
2.5000000000000004
works!


In [86]:
#sample parameters to ensure it is working

yhats = np.array([[1,2,3], [2,3,4], [3,4,5], [4,5,6]])
Cs = np.array([[[1,2,3],[1,2,3], [4,5,6]],[[1,1,1], [1,1,1], [1,1,1]]]) # 2 time steps, d = 3
p=2
d = 3

def cost(t):
    def func (y):
        #TODO: switching cost function
        # each element in decisions is a d x 1 vector, and C is dxd (for each time step)

        decisions = yhats[t - p : t]

        summ = np.zeros(d)
        for i in range(p):
            C = Cs[i]
            summ += np.matmul(C, decisions[p-i-1].T) #TODO: check
        norm = np.linalg.norm(y - summ)
        return (norm**2)/2
    return func

In [109]:
''' Test of the cost function (c)'''
func = cost(3)
print(func(np.array([1,2,3]))) #should be 3434.5 (works!)
print(func(np.array([34, 35, 70]))) #should be 1 (works!)
print("works!")

3434.5
1.0000000000000002
works!


In [146]:
# precondition: v_t must be a numpy array
l1 = 0.5
l2 = 0.25
def totalCost(fun, v_t, t):
    def func(y):
        return fun(y) + l1*cost(t)(y) + l2 * dist(v_t)(y)
    return func

In [113]:
func = totalCost(h, np.array([34,35,69]), t=3)
print(func(np.array([34,35,70]))) # should be 7,281 + 0.5 + (0.5*.25) = 7281.625
print("works!")

7281
7281.625
works!


In [136]:
''' Optimizers testing '''
# Find the d x 1 vector y that minimizes the function parameter
# TODO: change to convex optimizer
def _findMin(func):
    x0 = np.random.rand(d)
    res = minimize(func, x0, method='BFGS', options={'disp':False})
    return np.array(res.x)

In [135]:
print(_findMin(h)) # should print near 0's in a dx1 vector (works)

def h2(x):
    return np.sum((x-np.array([1,2,3]))**2)-1

print(_findMin(h2)) # should be around [1,2,3] (works)

print("works!")

[-1.04524355e-08 -4.77960365e-09 -4.77127775e-09]
[1.00000014 1.99999996 3.00000006]
works!


In [147]:
# subroutine for ROBD and optimistic ROBD
def robdSub(fun, t):
    vel = _findMin(fun)

    # below line: find minimum of entire expression with respect to y
    out = _findMin(totalCost(fun, vel, t))
    return out

In [153]:
''' Testing of Algorithm 1 (robd subroutine)'''
t = 3
fun = h2
print(robdSub(fun, t))
#v_t = [1,2,3] which means 

f = totalCost(fun, np.array([1,2,3]), 3)
print(f(np.array([7,8, 15])))
print(f(robdSub(fun, t))) # this is the min!
print(f(np.array([1,1,1])))
print("works!")

[ 7.18182189  7.99999977 15.36363985]
1404.2499999999998
1404.02272727274
1807.6250000000002
works!


In [161]:
lam = 0.5
def doubleFunc(h_t, t):
    def func(params):
        y, v = params
        return h_t(y - v) + lam * cost(t)(y)
    return func

In [174]:
fun = doubleFunc(h, 3)
print(0.5*cost(3)(np.array([1,2,3])))   # should print 1717.25
print(fun(np.array([[1,2,3], [1,2,4]]))) # should print 1718.25
print("works!")

1717.25
1718.25
works!


In [202]:
def constraint(omega_t):
    def func(params):
        y, v = params
        if tuple(v) in omega_t: #TODO: flesh out based on omega_t structure
            return 0
        return .1 #return a non-zero element
    return func

In [203]:
'''Constraint test'''
arrays = np.array([[1,2,3], [2,3,4], [3,4,5]])
omega_t = {tuple(row) for row in arrays}
print(omega_t)
fun = constraint(omega_t)
print(fun(np.array([[1,1,3], [1,2,3]]))) # should print 0   (found)
print(fun(np.array([[1,1,3], [1,2,6]]))) # should print 0.1 (not found)
print("works!")

{(3, 4, 5), (2, 3, 4), (1, 2, 3)}
0
0.1
works!


In [175]:
''' Hard part: double optimizer '''
#TODO: need to really check/test this
def findSetMin(function, omega_t):
    x0 = np.random.rand(d, 2)
    # constraint: must be in the set omega_t
    cons = ({'type': 'eq', 'fun': constraint(omega_t)})

    result = minimize(function, x0, method = 'SLSQP', constraints=cons)
    if result.success:
        fitted_params = np.array(result.x[:, 1]) #want to return v?
        return fitted_params
    else:
        raise ValueError(result.message)